In [7]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient
try:
    credential=DefaultAzureCredential()
    credential.get_token('https://management.azure.com/.default')
except Exception as ex:
    credential=InteractiveBrowserCredential()
mlclient=MLClient.from_config(credential)


Found the config file in: /config.json


In [1]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

#list the dataassets
for ds in mlclient.data.list():
    print(ds.name)


NameError: name 'mlclient' is not defined

In [32]:
import pandas as pd 
data=mlclient.data.get('diabetes-dataasset-hyperparam', version=1)
data=pd.read_csv(data.path)

In [51]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [53]:
sc.fit_transform(data[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin'
                           , 'BMI', 'DiabetesPedigreeFunction', 'Age']])

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [54]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
data[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin'
                           , 'BMI', 'DiabetesPedigreeFunction', 'Age']]=sc.fit_transform(data[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin'
                           , 'BMI', 'DiabetesPedigreeFunction', 'Age']])

In [55]:
data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0.639947,0.848324,0.149641,0.907270,-0.692891,0.204013,0.468492,1.425995,1
1,-0.844885,-1.123396,-0.160546,0.530902,-0.692891,-0.684422,-0.365061,-0.190672,0
2,1.233880,1.943724,-0.263941,-1.288212,-0.692891,-1.103255,0.604397,-0.105584,1
3,-0.844885,-0.998208,-0.160546,0.154533,0.123302,-0.494043,-0.920763,-1.041549,0
4,-1.141852,0.504055,-1.504687,0.907270,0.765836,1.409746,5.484909,-0.020496,1
...,...,...,...,...,...,...,...,...,...
763,1.827813,-0.622642,0.356432,1.722735,0.870031,0.115169,-0.908682,2.532136,0
764,-0.547919,0.034598,0.046245,0.405445,-0.692891,0.610154,-0.398282,-0.531023,0
765,0.342981,0.003301,0.149641,0.154533,0.279594,-0.735190,-0.685193,-0.275760,0
766,-0.844885,0.159787,-0.470732,-1.288212,-0.692891,-0.240205,-0.371101,1.170732,1


In [ ]:
import os
folder='src1'
os.makedirs(folder, exist_ok=True)

In [ ]:
%%writefile $folder/data_prep.py

import pandas as pd 
from sklearn.preprocessing import StandardScaler
from pathlib import Path
from argparse import ArgumentParser
def prep(args):
    df=pd.read_csv(args.input)
    df=df.dropna()
    sc=StandardScaler()
    #df[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
    #   'BMI', 'DiabetesPedigreeFunction', 'Age']]
  
    df[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin'
                           , 'BMI', 'DiabetesPedigreeFunction', 'Age']]=sc.fit_transform(df[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin'
                           , 'BMI', 'DiabetesPedigreeFunction', 'Age']])
    return df
def arguments():
    parser=ArgumentParser()
    parser.add_argument('--input',dest='input',type=str)
    parser.add_argument('--output',dest='output', type=str)
    args=parser.parse_args()
    return args
if __name__=='__main__':
    args=arguments()
    x=prep(args)
    pd.DataFrame(x).to_csv((Path(args.output)/'cleaned_diabetes.csv'), index=False)




Overwriting src1/data_prep.py


In [74]:
%%writefile $folder/train.py
import pandas as pd 
from sklearn.preprocessing import StandardScaler
from pathlib import Path
from argparse import ArgumentParser
def prep(args):
    df=pd.read_csv(args.input)
    df=df.dropna()
    sc=StandardScaler()
    #df[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
    #   'BMI', 'DiabetesPedigreeFunction', 'Age']]
  
    df[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin'
                           , 'BMI', 'DiabetesPedigreeFunction', 'Age']]=sc.fit_transform(df[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin'
                           , 'BMI', 'DiabetesPedigreeFunction', 'Age']])
    return df
def arguments():
    parser=ArgumentParser()
    parser.add_argument('--input',dest='input',type=str)
    parser.add_argument('--output',dest='output', type=str)
    args=parser.parse_args()
    return args
if __name__=='__main__':
    args=arguments()
    x=prep(args)
    pd.DataFrame(x).to_csv((Path(args.output)/'cleaned_diabetes.csv'), index=False)


Writing $folder/train.py


FileNotFoundError: [Errno 2] No such file or directory: '$folder/train.py'

In [56]:
%%writefile prep-data.yml

$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: prep_data
display_name: Prepare Data
version: 1
type: command 
inputs: 
     input_data:
        type: uri_file
outputs:
     output_data:
        type: uri_folder
code: /home/azureuser/cloudfiles/code/src/scripts/
environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest
command: >-
    python data_prep.py --input ${{inputs.input_data}} --output ${{outputs.output_data}}
    

Overwriting prep-data.yml


In [57]:
%%writefile train-data.yml
$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: train_model
display_name: Train Model
version: 1
type: command 
inputs: 
     training_data:
        type: uri_folder
     max_depth:
        type: number
        default: 2
     min_samples:
        type: number
        default: 2
     test_size:
        type: number
        default: 0.2
     random_state:
        type: number
        default: 23
outputs:
     model_output:
         type: mlflow_model
code: /home/azureuser/cloudfiles/code/src/scripts/
environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest
command: >-
    python train.py --input ${{inputs.training_data}} --max_depth ${{inputs.max_depth}} --min_samples ${{inputs.min_samples}} --test_size ${{inputs.test_size}} --random_state ${{inputs.random_state}}
    --model_output ${{outputs.model_output}}
    


Overwriting train-data.yml


In [58]:
# Assuming the corrected YAML file is available at the same location
from azure.ai.ml import load_component
prep_data=load_component('prep-data.yml')
train_data=load_component('train-data.yml')

from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.dsl import pipeline 

@pipeline()
def pipeline(pipe_inputs):
    clean_data=prep_data(input_data=pipe_inputs)
    train_model=train_data(training_data=clean_data.outputs.output_data, max_depth=2, min_samples=2, test_size=0.2, random_state=23)
    return {
        'pipeline_job_transformed_data': clean_data.outputs.output_data,
        'pipeline_job_trained_model': train_model.outputs.model_output
    }

In [59]:
pipeline_job=pipeline(Input(type=AssetTypes.URI_FILE, path='azureml:diabetes-dataasset-hyperparam:1'))

In [60]:
pipeline_job.settings.default_compute = 'satyakebakshi951'
pipeline_job.outputs.pipeline_job_transformed_data.mode='upload'
pipeline_job.outputs.pipeline_job_trained_model.mode='upload'

In [61]:
pipeline_submit=mlclient.jobs.create_or_update(pipeline_job, experiment_name='diabetes_pipeline')
pipeline_submit

Uploading scripts (0.08 MBs): 100%|██████████| 78099/78099 [00:00<00:00, 500283.99it/s]


Uploading scripts (0.08 MBs): 100%|██████████| 78352/78352 [00:00<00:00, 472745.16it/s]




Experiment,Name,Type,Status,Details Page
diabetes_pipeline,frank_kale_vh8q5yqgx2,pipeline,Preparing,Link to Azure Machine Learning studio
